In [95]:
from langchain_google_genai import ChatGoogleGenerativeAI,GoogleGenerativeAIEmbeddings
from langchain_core.prompts import ChatPromptTemplate
from langgraph.graph import StateGraph, MessagesState,START,END
from langgraph.graph.message import add_messages
from langgraph.prebuilt import create_react_agent
from pydantic import BaseModel, Field
from typing_extensions import Annotated, Literal,TypedDict, List

from dotenv import load_dotenv
import os
load_dotenv()

True

In [92]:
try:
    llm = ChatGoogleGenerativeAI(
        model="gemini-1.5-flash",
        temperature=0.3)
    response = llm.invoke("Are you ready to help me")
    print(response.content)
except Exception as e:
    print(f"Error initializing LLM: {e}")
    

Yes, I'm ready to help you.  Please tell me what you need.


## Document Load

In [16]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_chroma import Chroma
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import CharacterTextSplitter, RecursiveCharacterTextSplitter

In [80]:
file_path = 'data/cover_letter.pdf'
loader = PyPDFLoader(file_path,mode='single')
docs=loader.load()
spliter = CharacterTextSplitter(chunk_size=300, chunk_overlap=0,separator="\n")
texts = spliter.split_documents(docs)

In [72]:
texts

[Document(metadata={'producer': 'Skia/PDF m140 Google Docs Renderer', 'creator': 'PyPDF', 'creationdate': '', 'title': 'Cover letter for apexanalytix', 'source': 'data/cover_letter.pdf', 'total_pages': 2}, page_content="Natdanai  Intraraksa  \nAI  engineer  /  Data  Scientist   3217  w  Bertaue  Av.  Chicago,  IL  60618  \n Intraraksa@gmail.com  \n773-876-2897\n \n June  29,  2025  \nHiring  manager  \nApexanalytix  .Inc  \n I  am  writing  to  express  my  strong  interest  in  the  AI  Engineer  \nposition.\n \nWith\n \nover\n \nfive\n \nyears\n \nof\n \nexperience\n \nin\n \nAI\n \nengineering\n \nand\n \ndata\n \nscience,\n \nalong\n \nwith\n \na\n \ndecade\n \nof\n \nexperience\n \nas\n \na\n \nmanufacturing\n \nengineer,\n \nI\n \nbring\n \na\n \nwell-rounded\n \nbackground\n \nin\n \nboth\n \npractical\n \nengineering\n \nand\n \ncutting-edge\n \nAI\n \nsolution\n \ndevelopment.\n \nI\n \nrecently\n \nrelocated\n \nfrom\n \nThailand\n \nand\n \nam\n \neager\n \nto\n \ncontribute

In [81]:
# docs=loader.load()

In [82]:
# docs

In [83]:
# spliter = CharacterTextSplitter(chunk_size=300, chunk_overlap=0,separator="\n\n")
# texts = spliter.split_documents(docs)

In [ ]:
# text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0,is_separator_regex=True, separators=["\n\n", "\n", " ", ""])

In [54]:
# text_splitter.split_documents(docs)

## create vectorstore

In [84]:
# embedding 
embeddings = GoogleGenerativeAIEmbeddings(model='models/gemini-embedding-001')
vectorstore = Chroma.from_documents(texts,embedding=embeddings,collection_name="sample_collection",persist_directory="data/chroma_db")

## Create retrival tool

In [85]:
from langchain.tools.retriever import create_retriever_tool
retriver_tool = create_retriever_tool(
    retriever=vectorstore.as_retriever(),
    name="retriever_tool",
    description="This tool is used to retrieve relevant documents from the vector store based on the user's query.")

In [89]:
# a = retriver_tool.invoke({"query":"Which technic that I use to fine-tune the model?"})
# print(a)

In [108]:
## Grade Node
def generate_query_or_respond(state: MessagesState):
    """Call the model to generate a response based on the current state. Given
    the question, it will decide to retrieve using the retriever tool, or simply respond to the user.
    """
    response = (
        llm.bind_tools([retriver_tool]).invoke(state["messages"])
    )
    return {"messages": [response]}

In [100]:
def generate_query_or_respond(state: MessagesState):
    """Call the model to generate a response based on the current state. Given
    the question, it will decide to retrieve using the retriever tool, or simply respond to the user.
    """

    prompt = ''' Query relevant documents from the vector store based on the user's query.'''

    agent = create_react_agent(llm ,tools=[retriver_tool], prompt=prompt)
    response = agent.invoke({'messages':state["messages"]})
    return {"messages": [response]}

In [109]:
state = MessagesState()
state['messages'] = "Which framework that I use to build the agent?"

In [110]:
a = generate_query_or_respond(state)

In [113]:
a['messages'][0].pretty_print()

================================== Ai Message ==================================

The provided context only includes a `retriever_tool` function, which is a component that could be used within a larger agent framework.  There's no information about a complete agent framework.  To build an agent, you would need to choose a framework yourself, such as LangChain or similar.  The `retriever_tool` could then be integrated as a component within that framework.
